## 

In [ ]:
import json
import os
import pandas as pd
import requests
import time
import wandb
from datetime import datetime, timedelta
from dotenv import load_dotenv
from openai import OpenAI
from requests import JSONDecodeError, ConnectionError, ConnectTimeout


In [ ]:
# import wine.py from the scr folder
from config.config import *
from src.pairings import *
from src.recipe import *
from src.supermarket_and_fridge import *
from src.translation import *
from src.utils import *

In [ ]:
# Set day in config.py
print(f"Set day: {DAY}")
print(f"Set Model: {MODEL_GPT}")

In [ ]:
# Load env file
load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

In [ ]:
functions = [
    {
        "name": "get_current_date",
        "description": "Returns current the date, time, and weekday.",
        "parameters": { }
    },
    {
        "name": "is_public_holiday",
        "description": "Retrieves all public holidays of a German state. Default is 'Baden-Württemberg'.",
        "parameters": {
            "type": "object",
            "properties": {
                "state": {
                    "type": "string",
                    "description": "Name of state."
                },
            },
            "required": ["state"]
        }
    },
    {
        "name": "fruit_and_veg_offers",
        "description": "Retrieve supermarket offers for fruit and veg.",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "Date for which the offers should be retrieved. Offers always start on a Monday of a week. Format: 'YYYY-MM-DD'."
                },
            },
            "required": ["date"]
        }
    },
    {
        "name": "wine_selection_or_offers",
        "description": "Retrieve the wine selection from the supermarket. You can filter by wine type (red, rose, white) or offers on a specific date.",
        "parameters": {
            "type": "object",
            "properties": {
                "wine_type": {
                    "type": "string",
                    "description": "Filter the wine selection by wine type. Options: 'red', 'rose', 'white'. Default is None."
                },
                "date": {
                    "type": "string",
                    "description": "Date for which the offers should be retrieved. Format: 'YYYY-MM-DD'."
                },
            },
            "required": []
        }
    },
    {
        "name": "get_recipe_info",
        "description": "Retrieves detailed information about a recipe given its ID. This information includes the recipe's title and, optionally, cooking instructions, ingredient details such as names, IDs, and measurements, as well as preparation and cooking times.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipe_id": {
                    "type": "integer",
                    "description": "The unique identifier of the recipe."
                },
                "list_keys": {
                    "oneOf": [
                        {
                            "type": "null"
                        },
                        {
                            "type": "array",
                            "items": {
                                "type": "string"
                            }
                        }
                    ],
                    "description": "Optional. A list of information you want to retrieve. Options include 'time' for total, preparation, and cooking times;'instructions' for cooking instructions; 'ingredients_id' for ingredient IDs; 'ingredients' for ingredient names (optionally include 'measurements' for quantities). Can be None."
                }

            },
            "required": ["recipe_id"]
        }
    },
    {
        "name": "find_recipe",
            "description": "Finds recipes based on the name, ingredients, dietary preferences, cuisines, or maximal preparation time.", # be more specific with desciption / rephrase function names
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Name or main ingredient of recipe."
                    },
                    "diet": {
                        "type": "string",
                        "description": "Dietary preferences. Options are: 'vegetarian', 'lacto vegetarian', 'ovo vegetarian', 'vegan', 'pescetarian', 'paleo', 'primal'."
                    },
                    "include_ingredients": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Ingredients that should be included."
                    },
                    "exclude_ingredients": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Ingredients that should not be contained in the recipe."
                    },
                    "type": {
                        "type": "string",
                        "description": "Type of meal. Default is 'main course'."
                    },
                    "cuisine": {
                        "type": "string",
                        "description": "Cuisine preferences. Options are: 'African', 'Asian', 'American', 'British', 'Cajun', 'Caribbean', 'Chinese', 'Eastern European', 'European', 'French', 'German', 'Greek', 'Indian', 'Irish', 'Italian', 'Japanese', 'Jewish', 'Korean', 'Latin American', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Nordic', 'Southern', 'Spanish', 'Thai', 'Vietnamese'."
                    },
                    "exclude_cuisine": {
                        "type": "string",
                        "description": "Cuisines which should be avoided. Options are: 'African', 'Asian', 'American', 'British', 'Cajun', 'Caribbean', 'Chinese', 'Eastern European', 'European', 'French', 'German', 'Greek', 'Indian', 'Irish', 'Italian', 'Japanese', 'Jewish', 'Korean', 'Latin American', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Nordic', 'Southern', 'Spanish', 'Thai', 'Vietnamese'"
                    },
                    "max_ready_time": {
                        "type": "integer",
                        "description": "Maximal time for preparation and cooking (in minutes)."
                    }
            },
            "required": ["query"]
        }
    },
    {
        "name": "translate",
        "description": "Translates German text to English.",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "Text in German that should be translated to English."
                },
            },
            "required": ["text"]
        }
    },
    {
        "name": "dish_pairing_for_wine",
        "description": "Suggests food pairings for a given wine, providing a general description of the wine and a list of dishes that pair well with it.",
        "parameters": {
            "type": "object",
            "properties": {
                "grape_name": {
                    "type": "string",
                    "description": "The wine for which to suggest food pairings. Please provide the grape variety for best results. "
                }
            },
            "required": ["grape_name"]
        }
    },
    {
        "name": "wine_pairing",
        "description": "Provides wine pairing suggestions for a specified dish. Returns a selection of suitable wines along with a detailed description and, when applicable, a specific product recommendation.",
        "parameters": {
            "type": "object",
            "properties": {
                "dish_name": {
                    "type": "string",
                    "description": "The name of the food or dish for which wine suggestions are desired."
                }
            },
            "required": ["dish_name"]
        }
    },
    {
        "name": "image_classification",
        "description": "Retrieves the name of a dish given the URL of an image.",
        "parameters": {
            "type": "object",
            "properties": {
                "image_url": {
                    "type": "string",
                    "description": "The URL of an image."
                }
            },
            "required": ["image_url"]
        }
    },
    {
        "name": "compute_shopping_list",
        "description": "Adds items to a shopping list. The resulting shopping list includes the quantity and unit of each item as well as their aisle in the supermarket.",
        "parameters": {
            "type": "object",
            "properties": {
                "items_list": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "A list of items to add to the shopping list. Each item should be a string in the format 'quantity unit item'.",
                }
            },
            "required": ["items_list"]
        }
    },
    {
        "name": "fridge_items",
        "description": "Retrieves a list of vegetables that are currently in the fridge.",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "The current date in the format 'YYYY-MM-DD'."
                }
            },
            "required": ["date"]
        }
    },
]

### NEW QUESTIONS  
Conversation starts here

In [ ]:
# manually set GPT model:
MODEL_GPT = "gpt-4-turbo"

# MODEL_GPT = "gpt-4"

In [ ]:
user_prompt =  "Using veggies that are currently on offer, please make a pasta recipe suggestion. Please add all ingredients to my shopping list. I prefer European, and Chinese cuisines but I am allergic to dairy products."

In [ ]:
messages = [
    {
        "role": "system",
        "content": "Please complete the task to the best of your ability and do not ask for additional information. All steps can be solved with the help of the provided functions, for example, take note of the 'translate' or 'get_current_day' functions. Please make use of them whenever possible. If you encounter any issues, please let me know."
    },
    {
        "role":"user",
        "content": user_prompt
    }
]

# Configuration for retries in conversation
attempt = 0 # Initialize attempt counter
MAX_RETRIES = 10 # Maximum number of retries for connection with the OpenAI API
MAX_ATTEMPTS = 10 # Maximum number of attempts for a function call
RETRY_BACKOFF_FACTOR = 2  # Exponential backoff factor
initial_retry_delay = 1  # Initial delay in seconds


client = OpenAI(api_key=OPENAI_API_KEY) # Initialize OpenAI API client

try:
    attempt_timeout = 0 # Initialize timeout attempt counter
    func_name = None  # Initialize func_name to handle it safely in exception blocks
    params = None  # Initialize params similarly
    while attempt_timeout < MAX_RETRIES:
        try:
            completion = client.chat.completions.create(model=MODEL_GPT, messages=messages, functions=functions, function_call="auto")
            break  # Break the loop if the call is successful
        except requests.Timeout:
            attempt_timeout += 1
            if attempt_timeout == MAX_RETRIES:
                raise  # Re-raise the timeout exception if max retries reached
            time.sleep(initial_retry_delay * (RETRY_BACKOFF_FACTOR ** (attempt_timeout - 1)))  # Exponential backoff
    
    # Loop to handle function calls until completion
    while completion.choices[0].finish_reason == "function_call" and attempt < MAX_ATTEMPTS:
        params = json.loads(completion.choices[0].message.function_call.arguments) # Get function parameters
        func_name = completion.choices[0].message.function_call.name # Get function name
        chosen_function = eval(func_name)  # Get function object from function name
        try:
            function_result = chosen_function(**params) # Call the function with parameters
            
            messages.append({
                "role": "function",
                "name": func_name,
                "content": f"Function: {func_name}, Parameters: {json.dumps(params)}, Result: {str(function_result)}",
            })
            attempt = 0 # reset attempt counter
            completion = client.chat.completions.create(model=MODEL_GPT, messages=messages, functions=functions, function_call="auto")
        except Exception as e:
            attempt += 1
            error_message = f"Error in function {func_name} with parameters {json.dumps(params)}: {str(e)}"
            messages.append({
                "role": "error",
                "content": error_message
            })
            if attempt < MAX_ATTEMPTS:
                completion = client.chat.completions.create(model=MODEL_GPT, messages=messages, functions=functions, function_call="auto")
                params = json.loads(completion.choices[0].message.function_call.arguments) # update parameters
                func_name = completion.choices[0].message.function_call.name # update function name
                chosen_function = eval(func_name)  # update chosen_function

            if attempt >= MAX_ATTEMPTS:
                break  # Break after max attempts
    # Append the last message
    messages.append({"role": "assistant", "content": completion.choices[0].message.content})

# except json.JSONDecodeError as json_error:
#     # Handle invalid JSON errors
#     error_message = f"JSON error: {str(json_error)}"
#     messages.append({
#         "role": "json_error", "name": func_name, "content": error_message,
#     })
except requests.Timeout:
    # Handle timeout errors
    error_message = "Timeout error"
    messages.append({
        "role": "timeout",
        "content": error_message
    })
except Exception as e:
    if func_name is not None:
        error_message = f"Error in function {func_name} with parameters {json.dumps(params)}: {str(e)}"
    else:
        error_message = f"Error: {str(e)}"
    messages.append({
        "role": "error",  
        "content": error_message
    })


In [ ]:
# Full conversation
messages